In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configuración inicial
np.random.seed(42)  # Para reproducibilidad
dias = 365  # 1 año de datos
fecha_inicio = datetime(2025, 1, 1)

# 1. Datos principales de generación y consumo
def generar_datos_fotovoltaicos(dias):
    fechas = [fecha_inicio + timedelta(days=i) for i in range(dias)]

    datos = {
        'Fecha': fechas,
        'Radiacion_kWh_m2': np.random.normal(loc=5.0, scale=1.5, size=dias).clip(2, 8),  # Radiación solar
        'Temp_ambiente_C': np.random.normal(loc=25, scale=5, size=dias).clip(10, 40),    # Temperatura
        'Horas_pico_sol': np.random.uniform(3.5, 6.5, size=dias),                        # Horas de sol pico
        'Eficiencia_paneles': np.random.normal(loc=78, scale=3, size=dias).clip(70, 85)  # Eficiencia %
    }

    df = pd.DataFrame(datos)

    # Cálculo de energía generada (depende de radiación, horas pico y eficiencia)
    df['Energia_generada_kWh'] = (
        df['Radiacion_kWh_m2'] *
        df['Horas_pico_sol'] *
        1000 *  # Suponemos una planta de 1000 m2 (1 MWp aprox)
        (df['Eficiencia_paneles'] / 100) *
        (1 - 0.005 * (df['Temp_ambiente_C'] - 25))  # Pérdidas por temperatura
    ).round(1)

    # Añadir estacionalidad (más generación en verano)
    df['Energia_generada_kWh'] = df.apply(
        lambda x: x['Energia_generada_kWh'] * (1 + 0.3 * np.sin((x['Fecha'].timetuple().tm_yday / 365) * 2 * np.pi)),
        axis=1
    ).round(1)

    # Generar datos de consumo (correlacionado parcialmente con generación)
    df['Consumo_total_kWh'] = (
        df['Energia_generada_kWh'] *
        np.random.uniform(0.7, 1.3, size=dias) *
        (1 - 0.2 * np.sin((df['Fecha'].dt.month - 1) / 12 * 2 * np.pi))  # Menor consumo en verano
    ).round(1)

    # Autoconsumo y excedentes
    df['Autoconsumo_kWh'] = np.minimum(df['Energia_generada_kWh'], df['Consumo_total_kWh'] * 0.6)
    df['Excedentes_kWh'] = df['Energia_generada_kWh'] - df['Autoconsumo_kWh']

    # Pérdidas por mantenimiento (eventos aleatorios)
    df['Evento_mantenimiento'] = ''
    df['Perdida_energia_kWh'] = 0

    for i in range(random.randint(5, 10)):  # Entre 5 y 10 eventos al año
        dia = random.randint(0, dias-1)
        df.loc[dia, 'Perdida_energia_kWh'] = random.uniform(50, 300)
        df.loc[dia, 'Evento_mantenimiento'] = random.choice([
            'Limpieza paneles', 'Fallo inversor', 'Reparación cableado',
            'Mantenimiento preventivo', 'Actualización software'
        ])
        # Ajustar generación ese día
        df.loc[dia, 'Energia_generada_kWh'] -= df.loc[dia, 'Perdida_energia_kWh']

    return df

# Generar los datos
df_solar = generar_datos_fotovoltaicos(dias)

# 2. Datos de predicción (modelo simplificado)
def generar_predicciones(df):
    meses = df.groupby(df['Fecha'].dt.month).agg({
        'Energia_generada_kWh': 'sum'
    }).reset_index()

    meses['Energia_predicha_kWh'] = meses['Energia_generada_kWh'] * np.random.uniform(0.95, 1.05, size=12)
    meses['Error_prediccion_pct'] = (
        (meses['Energia_predicha_kWh'] - meses['Energia_generada_kWh']) /
        meses['Energia_generada_kWh'] * 100
    ).round(1)

    meses.rename(columns={'Fecha': 'Mes'}, inplace=True)
    return meses

df_predicciones = generar_predicciones(df_solar)

# 3. Datos horarios (para gráfica de distribución diaria)
def generar_datos_horarios():
    horas = pd.DataFrame({'Hora': range(24)})

    # Curva típica de generación solar (campana centrada en 12:00)
    horas['Generacion_kWh'] = (100 * np.exp(-0.5 * ((horas['Hora'] - 12) ** 2 / 4)).round(1))

    # Consumo típico (picos mañana y tarde)
    horas['Consumo_kWh'] = (
        50 + 30 * np.sin(horas['Hora'] / 24 * 2 * np.pi) +
        20 * np.sin(3 * horas['Hora'] / 24 * 2 * np.pi)
    ).round(1)

    return horas

df_horario = generar_datos_horarios()

# Guardar en archivos CSV
df_solar.to_csv('datos_planta_solar.csv', index=False)
df_predicciones.to_csv('predicciones_solar.csv', index=False)
df_horario.to_csv('consumo_horario.csv', index=False)

print("✅ Base de datos generada exitosamente!")
print(f"- Registros diarios: {len(df_solar)}")
print(f"- Meses predichos: {len(df_predicciones)}")
print(f"- Perfil horario: {len(df_horario)} horas")

✅ Base de datos generada exitosamente!
- Registros diarios: 365
- Meses predichos: 12
- Perfil horario: 24 horas


/tmp/ipython-input-8-2003443081.py:57: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '144.05707143393033' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [9]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import os

# Crear carpeta para guardar gráficas si no existe
if not os.path.exists('graficas'):
    os.makedirs('graficas')

# 1. Cargar datos generados (o usar los del código anterior)
df_solar = pd.read_csv('datos_planta_solar.csv', parse_dates=['Fecha'])
df_pred = pd.read_csv('predicciones_solar.csv')
df_horario = pd.read_csv('consumo_horario.csv')

# 2. Análisis de datos (funciones reutilizables)
def analizar_eficiencia(df):
    """Analiza la eficiencia vs temperatura"""
    df['Mes'] = df['Fecha'].dt.month_name()
    return df.groupby('Mes').agg({
        'Eficiencia_paneles': 'mean',
        'Temp_ambiente_C': 'mean'
    }).reset_index()

df_eficiencia = analizar_eficiencia(df_solar)

# 3. Generar todas las gráficas en HTML
def generar_graficas():
    # Gráfica 1: Producción vs Consumo mensual
    df_mensual = df_solar.resample('M', on='Fecha').sum().reset_index()
    fig1 = make_subplots(specs=[[{"secondary_y": True}]])

    fig1.add_trace(
        go.Bar(x=df_mensual['Fecha'], y=df_mensual['Energia_generada_kWh'],
               name='Generación', marker_color='#18a3cf'),
        secondary_y=False
    )

    fig1.add_trace(
        go.Scatter(x=df_mensual['Fecha'], y=df_mensual['Consumo_total_kWh'],
                   name='Consumo', line=dict(color='#e24d0d', width=3)),
        secondary_y=True
    )

    fig1.update_layout(
        title='Generación vs Consumo Mensual',
        xaxis_title='Mes',
        yaxis_title='Energía Generada (kWh)',
        yaxis2_title='Energía Consumida (kWh)',
        template='plotly_white'
    )
    fig1.write_html('graficas/produccion_consumo.html')

    # Gráfica 2: Eficiencia de paneles
    fig2 = px.scatter(
        df_eficiencia,
        x='Temp_ambiente_C',
        y='Eficiencia_paneles',
        color='Mes',
        size=[10]*12,
        title='Eficiencia de Paneles vs Temperatura',
        labels={'Temp_ambiente_C': 'Temperatura (°C)',
                'Eficiencia_paneles': 'Eficiencia (%)'},
        color_discrete_sequence=px.colors.sequential.Blues_r
    )
    fig2.write_html('graficas/eficiencia_paneles.html')

    # Gráfica 3: Distribución horaria
    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(
        x=df_horario['Hora'],
        y=df_horario['Generacion_kWh'],
        name='Generación',
        line=dict(color='#5bd1e7', width=3)
    ))
    fig3.add_trace(go.Scatter(
        x=df_horario['Hora'],
        y=df_horario['Consumo_kWh'],
        name='Consumo',
        line=dict(color='#39badb', width=3, dash='dot')
    ))
    fig3.update_layout(
        title='Distribución Horaria Típica',
        xaxis_title='Hora del día',
        yaxis_title='Energía (kWh)',
        showlegend=True
    )
    fig3.write_html('graficas/distribucion_horaria.html')

    # Gráfica 4: Matriz de correlación
    corr_matrix = df_solar[[
        'Energia_generada_kWh', 'Radiacion_kWh_m2',
        'Temp_ambiente_C', 'Horas_pico_sol'
    ]].corr()

    fig4 = px.imshow(
        corr_matrix,
        labels=dict(x="Variable", y="Variable", color="Correlación"),
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        color_continuous_scale=px.colors.diverging.RdBu_r,
        title='Matriz de Correlación'
    )
    fig4.write_html('graficas/matriz_correlacion.html')

    # Gráfica 5: Tendencia anual
    df_solar['Mes'] = df_solar['Fecha'].dt.month
    df_anual = df_solar.groupby('Mes').agg({
        'Energia_generada_kWh': 'sum',
        'Consumo_total_kWh': 'sum'
    }).reset_index()

    fig5 = px.line(
        df_anual,
        x='Mes',
        y=['Energia_generada_kWh', 'Consumo_total_kWh'],
        title='Tendencia Anual de Generación y Consumo',
        labels={'value': 'Energía (kWh)', 'variable': 'Tipo'},
        color_discrete_sequence=['#18a3cf', '#39badb']
    )
    fig5.write_html('graficas/tendencia_anual.html')

    # Gráfica 6: Predicción vs Real
    fig6 = go.Figure()
    fig6.add_trace(go.Bar(
        x=df_pred['Mes'],
        y=df_pred['Energia_generada_kWh'],
        name='Real',
        marker_color='#7ce8f3'
    ))
    fig6.add_trace(go.Scatter(
        x=df_pred['Mes'],
        y=df_pred['Energia_predicha_kWh'],
        name='Predicción',
        line=dict(color='#212492', width=3)
    ))
    fig6.update_layout(
        title='Predicción vs Generación Real',
        xaxis_title='Mes',
        yaxis_title='Energía (kWh)',
        barmode='group'
    )
    fig6.write_html('graficas/prediccion_consumo.html')

# Ejecutar generación de gráficas
generar_graficas()

print("📊 ¡Gráficas generadas con éxito!")
print(f"📍 Ubicación: {os.path.abspath('graficas')}")
print("\nGráficas creadas:")
print("- produccion_consumo.html")
print("- eficiencia_paneles.html")
print("- distribucion_horaria.html")
print("- matriz_correlacion.html")
print("- tendencia_anual.html")
print("- prediccion_consumo.html")

/tmp/ipython-input-9-1446211890.py:32: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



📊 ¡Gráficas generadas con éxito!
📍 Ubicación: /content/graficas

Gráficas creadas:
- produccion_consumo.html
- eficiencia_paneles.html
- distribucion_horaria.html
- matriz_correlacion.html
- tendencia_anual.html
- prediccion_consumo.html


In [10]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import os

# Configuración estética
COLOR_GENERACION = '#18a3cf'
COLOR_CONSUMO = '#39badb'
COLOR_PREDICCION = '#e24d0d'
COLOR_RESIDUAL = '#7ce8f3'

# 1. Datos sintéticos (simulando 1 año de operación)
np.random.seed(42)
meses = pd.date_range('2025-01-01', periods=12, freq='M')
generacion = np.array([4200, 4500, 5200, 5800, 6200, 6500, 6400, 6100, 5600, 5000, 4400, 4000]) * np.random.uniform(0.95, 1.05, 12)
consumo = generacion * np.random.uniform(0.7, 0.9, 12) * np.array([1.1, 1.0, 0.9, 0.85, 0.8, 0.75, 0.78, 0.82, 0.88, 0.95, 1.0, 1.05])

df_mensual = pd.DataFrame({
    'Mes': meses.strftime('%Y-%m'),
    'Generacion_kWh': generacion,
    'Consumo_kWh': consumo,
    'Radiacion_prom': [4.2, 4.5, 5.3, 5.8, 6.2, 6.5, 6.4, 6.1, 5.6, 5.0, 4.4, 4.0],
    'Temp_prom': [22, 23, 25, 27, 29, 31, 30, 29, 27, 25, 23, 22]
})

# 2. Gráfica Generación vs Consumo por Mes (Doble Eje)
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

fig1.add_trace(
    go.Bar(
        x=df_mensual['Mes'],
        y=df_mensual['Generacion_kWh'],
        name='Generación',
        marker_color=COLOR_GENERACION,
        opacity=0.8
    ),
    secondary_y=False
)

fig1.add_trace(
    go.Scatter(
        x=df_mensual['Mes'],
        y=df_mensual['Consumo_kWh'],
        name='Consumo',
        line=dict(color=COLOR_CONSUMO, width=3),
        mode='lines+markers',
        marker=dict(size=8)
    ),
    secondary_y=True
)

fig1.update_layout(
    title='<b>Generación vs Consumo Mensual</b>',
    xaxis_title='Mes',
    yaxis_title='Generación (kWh)',
    yaxis2_title='Consumo (kWh)',
    template='plotly_white',
    hovermode='x unified',
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            xref='paper',
            yref='paper',
            text='La generación supera el consumo en meses de verano',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

# 3. Gráfica Comparación por Meses (Heatmap)
df_comparacion = df_mensual.melt(id_vars=['Mes'], value_vars=['Generacion_kWh', 'Consumo_kWh'])
df_comparacion['Porcentaje'] = df_comparacion.groupby('Mes')['value'].transform(lambda x: x/x.sum()*100)

fig2 = px.bar(
    df_comparacion,
    x='Mes',
    y='value',
    color='variable',
    barmode='group',
    color_discrete_map={
        'Generacion_kWh': COLOR_GENERACION,
        'Consumo_kWh': COLOR_CONSUMO
    },
    text='Porcentaje',
    labels={'value': 'Energía (kWh)', 'variable': 'Tipo'},
    title='<b>Comparación Mensual Generación/Consumo</b>'
)

fig2.update_traces(
    texttemplate='%{text:.1f}%',
    textposition='outside'
)

fig2.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    yaxis=dict(range=[0, df_mensual[['Generacion_kWh', 'Consumo_kWh']].values.max() * 1.15])
)

# 4. Modelo Predictivo (Random Forest)
X = df_mensual[['Radiacion_prom', 'Temp_prom']]
y = df_mensual['Generacion_kWh']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Gráfica del Modelo Predictivo
fig3 = make_subplots(rows=1, cols=2, subplot_titles=('Predicciones vs Reales', 'Residuos del Modelo'))

# Subgráfica 1: Predicciones
fig3.add_trace(
    go.Scatter(
        x=y_test,
        y=y_pred,
        mode='markers',
        marker=dict(color=COLOR_PREDICCION, size=10),
        name='Predicciones',
        showlegend=False
    ),
    row=1, col=1
)

fig3.add_trace(
    go.Scatter(
        x=[y_test.min(), y_test.max()],
        y=[y_test.min(), y_test.max()],
        mode='lines',
        line=dict(color='grey', dash='dash'),
        name='Línea Perfecta'
    ),
    row=1, col=1
)

# Subgráfica 2: Residuos
residuos = y_test - y_pred
fig3.add_trace(
    go.Scatter(
        x=y_pred,
        y=residuos,
        mode='markers',
        marker=dict(color=COLOR_RESIDUAL, size=10),
        name='Residuos',
        showlegend=False
    ),
    row=1, col=2
)

fig3.add_hline(y=0, line_dash="dash", line_color="grey", row=1, col=2)

fig3.update_layout(
    title_text=f'<b>Modelo Predictivo de Generación (MAE: {mae:.0f} kWh | R²: {r2:.2f})</b>',
    showlegend=False,
    template='plotly_white'
)

fig3.update_xaxes(title_text='Valor Real (kWh)', row=1, col=1)
fig3.update_yaxes(title_text='Predicción (kWh)', row=1, col=1)
fig3.update_xaxes(title_text='Predicción (kWh)', row=1, col=2)
fig3.update_yaxes(title_text='Residuo (Real - Predicción)', row=1, col=2)

# Guardar gráficas
if not os.path.exists('graficas'):
    os.makedirs('graficas')

fig1.write_html('graficas/generacion_vs_consumo.html')
fig2.write_html('graficas/comparacion_meses.html')
fig3.write_html('graficas/modelo_predictivo.html')

print("✅ Gráficas generadas exitosamente:")
print(f"1. generacion_vs_consumo.html - Comparación mensual con doble eje")
print(f"2. comparacion_meses.html - Análisis porcentual por mes")
print(f"3. modelo_predictivo.html - Modelo RF con métricas (MAE: {mae:.0f} kWh, R²: {r2:.2f})")

/tmp/ipython-input-10-918134808.py:18: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



✅ Gráficas generadas exitosamente:
1. generacion_vs_consumo.html - Comparación mensual con doble eje
2. comparacion_meses.html - Análisis porcentual por mes
3. modelo_predictivo.html - Modelo RF con métricas (MAE: 311 kWh, R²: 0.42)
